In [ ]:
#接着interp_and_cal_MasterD后续，旨在利用一些算法计算出最佳延迟年份，仅是对之前的结果进行简单的求一些统计值

import math
import pandas as pd
import scipy.stats
import numpy as np

def coordinate(x):
    degree = math.floor(x)
    minutes = round(((x - degree) * 100)/60,2)
    result = degree + minutes
    return result

pointsname = pd.read_excel('point(去除东山).xls', index_col=None, na_values=['NA'])
pointnumber = pointsname.loc[:,'区站号'].tolist()
pointname = pointsname.loc[:,'站名'].tolist()
pointprovince = pointsname.loc[:,'省份'].tolist()
pointlatitude = pointsname.loc[:,'纬度（度.分）'].map(lambda x: coordinate(x)).tolist() 
pointlongitude = pointsname.loc[:,'经度（度.分）'].map(lambda x: coordinate(x)).tolist()
pointaltitude = pointsname.loc[:,'观测场拔海高度（米）'].tolist()
numbername = dict(zip(pointnumber, pointname))
numberprovince = dict(zip(pointnumber, pointprovince))
numberlatitude = dict(zip(pointnumber, pointlatitude))
numberlongitude = dict(zip(pointnumber, pointlongitude))
numberaltitude = dict(zip(pointnumber, pointaltitude))

In [ ]:
def deleteNone(data):
    result=[]
    for x in data:
        if x == None:
            pass
        else:
            result.append(x)
    return result

def bestlagyear(corResult, pointcoordinate, delta): 
    
    Selectdata =corResult.loc[corResult["Latitude"] > pointcoordinate[0] - delta].loc[corResult["Latitude"] < pointcoordinate[0] + delta].loc[corResult["Longitude"] > pointcoordinate[1] - delta].loc[corResult["Longitude"] < pointcoordinate[1] + delta].reset_index(drop=True)
    
    LAG=[]
    pointsum= []
    Raver=[]
    Paver=[]
    Rstd=[]
    Pstd=[]
    
    for lagyear in list(range(0,corrConfig.get('Lag')+1)):
        Rdata= np.array(deleteNone(Selectdata.loc[:,'R_Lag_'+str(lagyear)+'_unit'].dropna(axis=0).tolist()))
        Pdata= np.array(deleteNone(Selectdata.loc[:,'P_Lag_'+str(lagyear)+'_unit'].dropna(axis=0).tolist()))
        
        LAG.append(lagyear)
        pointsum.append(Selectdata.shape[0])
        Raver.append(np.mean(Rdata))
        Paver.append(np.mean(Pdata))        
        Rstd.append(np.std(Rdata))
        Pstd.append(np.std(Pdata))
        
    dataresult = pd.DataFrame({'Lag(Unit)': LAG,
                               'Point_Sum': pointsum,
                               'R_Average':Raver,
                                'P_Average': Paver,
                                'R_Std': Rstd,
                                'P_Std':Pstd})
    config = pd.DataFrame({'Name':['Distance','Startyear','Endyear','Lag','Cave_Latitude','Cave_Longitude','Delta'],
                          'Data':[corrConfig.get('Distance'),corrConfig.get('Startyear'),corrConfig.get('Endyear'),corrConfig.get('Lag'),pointcoordinate[0],pointcoordinate[1],delta]})
        
    return [dataresult,Selectdata,config]

In [ ]:
Filename = 'LFD_1.xlsx' 
correlateData = pd.read_excel(r'corrStalg_Rainfall_'+Filename,'Data',  index_col=None, na_values=['NA'])
correlateConfig = pd.read_excel(r'corrStalg_Rainfall_'+Filename,'Config',  index_col=None, na_values=['NA'])

ConfigName = correlateConfig.loc[:,'Name'].tolist()
ConfigData = correlateConfig.loc[:,'Data'].tolist()
corrConfig = dict(zip(ConfigName, ConfigData))

In [ ]:
LagforSta = bestlagyear(corResult=correlateData, pointcoordinate=[25+round(11/60,2),112+round(49/60,2)], delta=2)#LFD [25+round(11/60,2),112+round(49/60,2)]
#Shihua [39+round(47/60,2),115+round(56/60,2)];  Dayu [33+round(8/60,2),106+round(18/60,2)];   Wanxiang  [33+round(19/60,2),105]


Selecteddatawriter = pd.ExcelWriter('BestLag_'+Filename)
LagforSta[0].to_excel(Selecteddatawriter,index=False, sheet_name='Result') 
LagforSta[1].to_excel(Selecteddatawriter,index=False, sheet_name='Selected_Data') 
LagforSta[2].to_excel(Selecteddatawriter,index=False, sheet_name='Config') 
Selecteddatawriter.save()

print(LagforSta[0])